In [81]:
import seaborn as sns
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
import itertools

In [82]:
resources = Path(os.path.abspath('../resources'))
cassanda_path = Path(os.path.abspath('../resources/cassanda'))
hive_path = Path(os.path.abspath('../resources/hive'))

print(resources)
print(cassanda_path)
print(hive_path)

/Users/Jumma/github_repo/github_api_extractor/resources
/Users/Jumma/github_repo/github_api_extractor/resources/cassanda
/Users/Jumma/github_repo/github_api_extractor/resources/hive


In [83]:
hive_2015 = pd.read_csv(hive_path/'Output_hive_2015Aug05_TestSmellDetection_1675797361331.csv')
hive_2016 = pd.read_csv(hive_path/'Output_hive_2016Jul06_TestSmellDetection_1675796997456.csv')
hive_2017 = pd.read_csv(hive_path/'Output_hive_2017Apr05_TestSmellDetection_1675796760046.csv')
hive_2018 = pd.read_csv(hive_path/'Output_hive_2018Dec07_TestSmellDetection_1675796163621.csv')
hive_2019 = pd.read_csv(hive_path/'Output_hive_2019Nov06_TestSmellDetection_1675795961152.csv')
hive_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   App                          15 non-null     object 
 1   TestClass                    15 non-null     object 
 2   TestFilePath                 15 non-null     object 
 3   ProductionFilePath           0 non-null      float64
 4   RelativeTestFilePath         15 non-null     object 
 5   RelativeProductionFilePath   0 non-null      float64
 6   NumberOfMethods              15 non-null     int64  
 7   Assertion Roulette           15 non-null     int64  
 8   Conditional Test Logic       15 non-null     int64  
 9   Constructor Initialization   15 non-null     int64  
 10  Default Test                 15 non-null     int64  
 11  EmptyTest                    15 non-null     int64  
 12  Exception Catching Throwing  15 non-null     int64  
 13  General Fixture       

In [84]:
test_semantic_smell = ['Assertion Roulette','Conditional Test Logic','Duplicate Assert']
issue_in_test_step = ['Exception Catching Throwing', 'General Fixture', 'EmptyTest', 'Redundant Assertion', 'Unknown Test', 'Constructor Initialization']
code_related = ['Magic Number Test', 'Print Statement', 'IgnoredTest', 'Verbose Test']
dependencies = ['Mystery Guest', 'Resource Optimism']
test_execution = ['Sensitive Equality', 'Sleepy Test', 'Sensitive Equality']
hive_2015.head(5)

,App,TestClass,TestFilePath,ProductionFilePath,RelativeTestFilePath,RelativeProductionFilePath,NumberOfMethods,Assertion Roulette,Conditional Test Logic,Constructor Initialization,...,Verbose Test,Sleepy Test,Eager Test,Lazy Test,Duplicate Assert,Unknown Test,IgnoredTest,Resource Optimism,Magic Number Test,Dependent Test
0,hive,TestCloudComputeService.java,/Users/Jumma/github_repo/hive/testutils/ptest2...,NaN,ptest/execution/context/TestCloudComputeServic...,NaN,5,0,0,0,...,0,0,NaN,NaN,0,0,0,0,4,0
1,hive,TestTestParser.java,/Users/Jumma/github_repo/hive/testutils/ptest2...,NaN,ptest/execution/conf/TestTestParser.java,NaN,6,0,0,0,...,0,0,NaN,NaN,0,0,0,1,3,0
2,hive,TestTestConfiguration.java,/Users/Jumma/github_repo/hive/testutils/ptest2...,NaN,ptest/execution/conf/TestTestConfiguration.java,NaN,3,2,0,0,...,0,0,NaN,NaN,0,0,0,0,3,0
3,hive,TestQFileTestBatch.java,/Users/Jumma/github_repo/hive/testutils/ptest2...,NaN,ptest/execution/conf/TestQFileTestBatch.java,NaN,4,2,0,0,...,0,0,NaN,NaN,0,0,0,0,3,0
4,hive,TestScripts.java,/Users/Jumma/github_repo/hive/testutils/ptest2...,NaN,ptest/execution/TestScripts.java,NaN,10,0,0,0,...,0,0,NaN,NaN,0,6,0,0,6,0


In [118]:
def clean_data(df_input: pd, wanted_test_smell: list, classification_test_smell: str, year: int):
    result_df = pd.DataFrame(columns=['Test_Smell_Classification_Type', 'Test_Smell', 'Number_Of_Test_Class','Total_Of_Test_Class', 'List_Of_Test_Class',  'Number_Of_Test_Smell', 'Total_Of_Test_Smell', 'Year'])
    df_input = df_input.dropna(axis='columns')
    test_smell_df = df_input.loc[:, 'Assertion Roulette':'Dependent Test']
    test_smell_df['TestClass'] = df_input['TestClass']
    #Test smell calculate part
    # test_smell_temp = test_smell_df.sum()
    # test_smell_temp = pd.Series(test_smell_temp)
    # test_smell_temp = pd.DataFrame({'Test_Smell':test_smell_temp.index, 'Number_Of_Test_Smell' : test_smell_temp}).reset_index(drop=True)
    # result_df = pd.concat([result_df, test_smell_temp], ignore_index=True)
    print(test_smell_df)

    for column in wanted_test_smell:
        test_smell = pd.DataFrame({'Test_Smell': [column]})
        temp_test_class = test_smell_df.loc[test_smell_df[column] > 0, 'TestClass'].tolist()
        list_of_test_class = pd.DataFrame({'List_Of_Test_Class': [temp_test_class]})
        number_of_test_class = pd.DataFrame({'Number_Of_Test_Class': [len(temp_test_class)]})
        Number_Of_Test_Smell = pd.DataFrame({'Number_Of_Test_Smell': [test_smell_df[column].sum()]})
        temp_list = pd.concat([test_smell, list_of_test_class, number_of_test_class, Number_Of_Test_Smell], axis=1)
        result_df = pd.concat([result_df,temp_list])

    result_df['Total_Of_Test_Class'] = len(list(dict.fromkeys(list(itertools.chain.from_iterable(result_df['List_Of_Test_Class'].tolist())))))
    result_df['Test_Smell_Classification_Type'] = classification_test_smell
    result_df['Year'] = year
    result_df['Total_Of_Test_Smell'] = result_df['Number_Of_Test_Smell'].sum()

    return result_df

    Assertion Roulette  Conditional Test Logic  Constructor Initialization  \
0                    0                       0                           0   
1                    0                       0                           0   
2                    2                       0                           0   
3                    2                       0                           0   
4                    0                       0                           0   
5                    0                       0                           0   
6                    1                       1                           0   
7                    0                       0                           0   
8                    2                       0                           0   
9                    2                       2                           0   
10                   6                       0                           0   
11                   0                       0                  

,Test_Smell_Classification_Type,Test_Smell,Number_Of_Test_Class,Total_Of_Test_Class,List_Of_Test_Class,Number_Of_Test_Smell,Total_Of_Test_Smell,Year
0,test_semantic_smell,Assertion Roulette,8,9,"[TestTestConfiguration.java, TestQFileTestBatc...",20,29,2015
0,test_semantic_smell,Conditional Test Logic,2,9,"[TestReportParser.java, TestJIRAService.java]",3,29,2015
0,test_semantic_smell,Duplicate Assert,3,9,"[TestLocalCommand.java, TestJIRAService.java, ...",6,29,2015


In [ ]:
#2015
test_semantic_smell_2015 = clean_data(hive_2015, test_semantic_smell, 'test_semantic_smell', 2015)
issue_in_test_step_2015 = clean_data(hive_2015, issue_in_test_step, 'test_semantic_smell', 2015)
code_related_2015 = clean_data(hive_2015, code_related, 'test_semantic_smell', 2015)
dependencies_2015 = clean_data(hive_2015, dependencies, 'test_semantic_smell', 2015)
test_execution_2015 = clean_data(hive_2015, test_execution, 'test_semantic_smell', 2015)
#2016
test_semantic_smell_2016 = clean_data(hive_2016, test_semantic_smell, 'test_semantic_smell', 2016)
issue_in_test_step_2016 = clean_data(hive_2016, issue_in_test_step, 'test_semantic_smell', 2016)
code_related_2016 = clean_data(hive_2016, code_related, 'test_semantic_smell', 2016)
dependencies_2016 = clean_data(hive_2016, dependencies, 'test_semantic_smell', 2016)
test_execution_2016 = clean_data(hive_2016, test_execution, 'test_semantic_smell', 2016)
#2017
test_semantic_smell_2017 = clean_data(hive_2017, test_semantic_smell, 'test_semantic_smell', 2017)
issue_in_test_step_2017 = clean_data(hive_2017, issue_in_test_step, 'test_semantic_smell', 2017)
code_related_2017 = clean_data(hive_2017, code_related, 'test_semantic_smell', 2017)
dependencies_2017 = clean_data(hive_2017, dependencies, 'test_semantic_smell', 2017)
test_execution_2017 = clean_data(hive_2017, test_execution, 'test_semantic_smell', 2017)
#2018
#2019

In [9]:
test_hive = hive_2015.dropna(axis='columns')
df = test_hive.loc[:, 'NumberOfMethods':'Dependent Test']
total =  df.sum()
total = pd.Series(total)
frame = {'Amount': total, 'Year': 2015}
result = pd.DataFrame(frame)
result.reset_index(inplace=True)
result = result.rename(columns={'index': 'Test_Smell'}, errors='raise')
result

,Test_Smell,Amount,Year
0,NumberOfMethods,74,2015
1,Assertion Roulette,20,2015
2,Conditional Test Logic,3,2015
3,Constructor Initialization,0,2015
4,Default Test,0,2015
5,EmptyTest,0,2015
6,Exception Catching Throwing,44,2015
7,General Fixture,22,2015
8,Mystery Guest,3,2015
9,Print Statement,0,2015


In [626]:
def total_test_smell(dataframe : pd.DataFrame, year):
    dataframe = dataframe.dropna(axis='columns')
    df = dataframe.loc[:, 'Assertion Roulette':'Dependent Test']
    total =  df.sum()
    total = pd.Series(total)
    frame = {'Amount': total, 'Year': year}
    result = pd.DataFrame(frame)
    result.reset_index(inplace=True)
    result = result.rename(columns={'index': 'Test_Smell_Type'}, errors='raise')
    result['Test_Smell_Total'] = result['Amount'].sum()
    return result

In [627]:
def total_test_method(dataframe : pd.DataFrame, year):
    dataframe = dataframe.dropna(axis='columns')
    df = dataframe.loc[:, 'NumberOfMethods':'Dependent Test']
    total =  df.sum()
    total = pd.Series(total)
    frame = {'Amount': total, 'Year': year}
    result = pd.DataFrame(frame)
    result.reset_index(inplace=True)
    result = result.rename(columns={'index': 'Test_Smell_Type'}, errors='raise')
    return result

    NumberOfMethods  Assertion Roulette  Conditional Test Logic  \
0                 5                   0                       0   
1                 6                   0                       0   
2                 3                   2                       0   
3                 4                   2                       0   
4                10                   0                       0   
5                 3                   0                       0   
6                 3                   1                       1   
7                 2                   0                       0   
8                 2                   2                       0   
9                 7                   2                       2   
10               11                   6                       0   
11                2                   0                       0   
12               10                   4                       0   
13                1                   1                       

KeyError: 'Ignored Test'

In [ ]:
testsmell2015 = total_test_method(hive_2015, 2015)
testsmell2016 = total_test_method(hive_2016, 2016)
testsmell2017 = total_test_method(hive_2017, 2017)
testsmell2018 = total_test_method(hive_2018, 2018)
testsmell2019 = total_test_method(hive_2019, 2019)
all = [testsmell2015,testsmell2016,testsmell2017,testsmell2018,testsmell2019]
df = pd.concat(all, axis=0)
df = df.loc[df['Test_Smell_Type'] == 'NumberOfMethods']
df = df.rename(columns={'Test_Smell_Type': 'Test_Method', 'Amount': 'Test_Method_Total'})
df = df.drop(['Test_Method'], axis=1)
df
# sns.barplot(x='Year', y='Amount', hue='Test_Smell_Type', data=df)

In [ ]:
total_hive2015 = total_test_smell(hive_2015, 2015)
total_hive2016 = total_test_smell(hive_2016, 2016)
total_hive2017 = total_test_smell(hive_2017, 2017)
total_hive2018 = total_test_smell(hive_2018, 2018)
total_hive2019 = total_test_smell(hive_2019, 2019)
all_list_hive = [total_hive2015,total_hive2016,total_hive2017,total_hive2018,total_hive2019]
all_list_hive_total = pd.concat(all_list_hive, axis=0)
all_list_hive_total = all_list_hive_total.reset_index(drop=True)
all_list_hive_total = all_list_hive_total.merge(df, on='Year', how='left')
all_list_hive_total
# all_list_hive_total = all_list_hive_total.rename({0: '2015',1: '2016',2: '2017',3: '2018',4: '2019'}, axis='columns', errors='raise')
# all_list_hive_total


In [ ]:
star_each_year = pd.DataFrame({
    'Year':[2015, 2016, 2017, 2018, 2019],
    'Star':[570, 870, 1200, 2130, 2760]
})
star_each_year
all_list_hive_total = all_list_hive_total.merge(star_each_year, on='Year', how='left')
all_list_hive_total

In [ ]:
all_list_hive_total['Density_test_smell'] = (all_list_hive_total['Test_Smell_Total']/all_list_hive_total['Test_Method_Total'])
all_list_hive_total['Density_each_test_smell'] = (all_list_hive_total['Amount']/all_list_hive_total['Test_Method_Total'])
all_list_hive_total
all_list_hive_total.to_csv('/Users/Jumma/github_repo/github_api_extractor/resources/dataframe/hive.csv')

In [ ]:
# create line chart with star
sns.set_style("whitegrid")
plt.figure(figsize=(10,6))
sns.lineplot(data=all_list_hive_total.set_index('Year').drop(['Amount', 'Density_test_smell', 'Density_each_test_smell'], axis=1), linewidth=2.5, dashes=False)
plt.title('Total Number of Test Smells and Test Methods by Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Amount', fontsize=14)
plt.show()

In [ ]:
# create line chart with star
sns.set_style("whitegrid")
plt.figure(figsize=(10,6))
ax1 = sns.lineplot(data=all_list_hive_total.set_index('Year'), linewidth=2.5 ,x='Year', y='Amount',hue='Test_Smell_Type', dashes=False)
plt.legend(bbox_to_anchor=(1.12, 1), loc='upper left', borderaxespad=0)
ax2 = ax1.twinx()
sns.lineplot(data=all_list_hive_total.set_index('Year'),linewidth=2.5 ,x='Year', y='Star', color='green')
plt.title('Test Smell Types by Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
# plt.ylabel('Amount', fontsize=14)


plt.show()

In [ ]:
# create line chart with star
sns.set_style("whitegrid")
plt.figure(figsize=(10,6))
ax1 = sns.lineplot(data=all_list_hive_total.set_index('Year'), linewidth=2.5 ,x='Year', y='Density_each_test_smell',hue='Test_Smell_Type', dashes=False)
plt.legend(bbox_to_anchor=(1.12, 1), loc='upper left', borderaxespad=0)
ax2 = ax1.twinx()
sns.lineplot(data=all_list_hive_total.set_index('Year'),linewidth=2.5 ,x='Year', y='Star', color='green')
plt.title('Test Smell density by Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
# plt.ylabel('Amount', fontsize=14)

plt.show()

In [ ]:
test_semantic_smell = ['Assertion Roulette','Conditional Test Logic','Duplicate Assert', 'Eager Test', 'Lazy Test']
test_semantic_smell = all_list_hive_total[~all_list_hive_total.Test_Smell_Type.str.contains('|'.join(test_semantic_smell)) == False]
test_semantic_smell['Classification'] = 'test semantic'
temp = test_semantic_smell.groupby('Year')['Amount'].sum()
test_semantic_smell = test_semantic_smell.merge(temp.rename('Classification_Test_Smell_Total'), left_on='Year', right_on='Year')
issue_in_test_step = ['Exception Handling', 'General Fixture', 'Empty Test', 'Redundant Assertion', 'Unknown Test']
issue_in_test_step = all_list_hive_total[~all_list_hive_total.Test_Smell_Type.str.contains('|'.join(issue_in_test_step)) == False]
issue_in_test_step['Classification'] = 'issue in test step'
temp = issue_in_test_step.groupby('Year')['Amount'].sum()
issue_in_test_step = issue_in_test_step.merge(temp.rename('Classification_Test_Smell_Total'), left_on='Year', right_on='Year')
code_related = ['Magic Number Test', 'Redundant Print', 'Ignored Test']
code_related = all_list_hive_total[~all_list_hive_total.Test_Smell_Type.str.contains('|'.join(code_related)) == False]
code_related['Classification'] = 'code related'
temp = code_related.groupby('Year')['Amount'].sum()
code_related = code_related.merge(temp.rename('Classification_Test_Smell_Total'), left_on='Year', right_on='Year')
test_execution = ['Sensitive Equality', 'Sleepy Test']
test_execution = all_list_hive_total[~all_list_hive_total.Test_Smell_Type.str.contains('|'.join(test_execution)) == False]
test_execution['Classification'] = 'test execution'
temp = test_execution.groupby('Year')['Amount'].sum()
test_execution = test_execution.merge(temp.rename('Classification_Test_Smell_Total'), left_on='Year', right_on='Year')
dependencies = ['Mystery Guest', 'Resource Optimism']
dependencies = all_list_hive_total[~all_list_hive_total.Test_Smell_Type.str.contains('|'.join(dependencies)) == False]
dependencies['Classification'] = 'dependencies'
temp = dependencies.groupby('Year')['Amount'].sum()
dependencies = dependencies.merge(temp.rename('Classification_Test_Smell_Total'), left_on='Year', right_on='Year')
list_Classification = [test_semantic_smell, issue_in_test_step, code_related, test_execution, dependencies]
list_Classification = pd.concat(list_Classification)
list_Classification = list_Classification.reset_index(drop=True)
list_Classification
# list_classification = ['test semantic', 'issue in test step', 'code related', 'test execution', 'dependencies']

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10,6))
ax1 = sns.lineplot(data=list_Classification.set_index('Year'), linewidth=2.5 ,x='Year', y='Classification_Test_Smell_Total' ,hue='Classification', dashes=False)
plt.legend(bbox_to_anchor=(1.12, 1), loc='upper left', borderaxespad=0)
ax2 = ax1.twinx()
sns.lineplot(data=all_list_hive_total.set_index('Year'),linewidth=2.5 ,x='Year', y='Star', color='gray')
plt.title('Classification test smell total by year, Hive', fontsize=16)
plt.xlabel('Year', fontsize=14)
# plt.ylabel('Amount', fontsize=14)

plt.show()

### Classification by Vahid Garousi 2018

In [ ]:
sns.set_style("whitegrid")
listClassification = [test_semantic_smell, issue_in_test_step, code_related, test_execution, dependencies]

for a in listClassification:
    plt.figure(figsize=(10,6))
    ax1 = sns.lineplot(data=a.set_index('Year'), linewidth=2.5 ,x='Year', y='Amount',hue='Test_Smell_Type', dashes=False)
    plt.legend(bbox_to_anchor=(1.12, 1), loc='upper left', borderaxespad=0)
    ax2 = ax1.twinx()
    sns.lineplot(data=all_list_hive_total.set_index('Year'),linewidth=2.5 ,x='Year', y='Star', color='green')
    plt.title('Total Test Smell by Year', fontsize=16)
    plt.xlabel('Year', fontsize=14)
    # plt.ylabel('Amount', fontsize=14)
    plt.show()

## Most discuss until 2018

In [ ]:
most_discuss = ['Assertion Roulette', 'Eager Test', 'General Fixture', 'Mystery Guest', 'Redundant Print', 'Redundant Assertion']
most_discuss = all_list_hive_total[~all_list_hive_total.Test_Smell_Type.str.contains('|'.join(most_discuss)) == False]

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(10,6))
ax1 = sns.lineplot(data=most_discuss.set_index('Year'), linewidth=2.5 ,x='Year', y='Amount',hue='Test_Smell_Type', dashes=False)
plt.legend(bbox_to_anchor=(1.12, 1), loc='upper left', borderaxespad=0)
ax2 = ax1.twinx()
sns.lineplot(data=all_list_hive_total.set_index('Year'),linewidth=2.5 ,x='Year', y='Star', color='green')
plt.title('Test Smell density by Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
# plt.ylabel('Amount', fontsize=14)
plt.show()

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10,6))

data = all_list_hive_total[(all_list_hive_total != 0).all(1)]

sns.kdeplot(data=data, x='Amount', y='Test_Method_Total', hue='Test_Smell_Type')
# plt.legend(loc='center right', title='test smell type',fontsize="x-large", bbox_to_anchor=(1.2, 0.5))
plt.show()
data
# datanew = all_list_hive_total[(all_list_hive_total != 0).all(1)]
# datanew

In [ ]:

# Create a scatter plot with Test_Smell_Total and Test_Method_Total
sns.set_style('white')
sns.scatterplot(x='Test_Smell_Total', y='Test_Method_Total', data=all_list_hive_total, alpha=0.5)

# Create a heatmap to show density in each year
sns.kdeplot(x='Test_Smell_Total', y='Test_Method_Total',  data=all_list_hive_total, cmap="Reds", shade=True, shade_lowest=False)

# Add labels and title
plt.xlabel('Test Smell Total')
plt.ylabel('Test Method Total')
plt.title('Density Plot of Test Smell Total vs. Test Method Total by Year')

plt.show()

In [ ]:
sns.jointplot(data=all_list_hive_total, x="Test_Method_Total", y="Test_Smell_Total", kind="kde", fill=True)

In [ ]:
sns.scatterplot(data=all_list_hive_total, x='Test_Method_Total', y='Test_Smell_Total', size='Density_test_smell', hue='Year')
plt.legend(bbox_to_anchor=(1.12, 1), loc='upper left', borderaxespad=0)


In [ ]:
# Group the data by Test Smell Type and sum the Amount column
grouped = all_list_hive_total.groupby('Year')

for year, data in grouped:
    # Get the sum of each test smell type for the current year
    sums = data.groupby('Test_Smell_Type')['Amount'].sum()

    # Create the pie chart
    colors = sns.color_palette("husl", 19)
    plt.pie(sums, autopct='%1.1f%%', radius=1.5, colors=colors)
    plt.legend(sums.index, loc="upper right",bbox_to_anchor=(2, 1), title=f'Test Smells in {year}')
    plt.show()

In [ ]:
def test_smell_show_inEachClass(column):
    plt.figure(figsize=(10,5))
    g = sns.scatterplot(data=hive_2015, x="TestClass", y=column)
    g.set_xticklabels(g.get_xticklabels(),rotation=90)
    plt.show()

In [ ]:
pivot_table = pd.pivot_table(all_list_hive_total, values='Amount', index=['Test_Smell_Type'], columns=['Year'], aggfunc='mean')
pivot_table

In [ ]:
g = sns.pairplot(data=all_list_hive_total, hue='Test_Smell_Type', x_vars=['Year'], y_vars=['Amount'], height=5)
for ax in g.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()


From graph pairplot above, You will see the Case that are always high is ***Magic Number*** and ***Exception Catching Throwing*** Seem like they won't try to fix those Test_Smell at all. At the hive project its seem like stable, but in the cassandar trend on those test smell is slightly high in each year(2018 look like they change all the code or migrate to use Junit5)

In [ ]:
sns.heatmap(pivot_table, cmap='coolwarm')

For this heatmap we change the style of graph to investigate more trend of this data. **In Hive project**, You can see the test smell are not decrease from year to year at all, **same as the cassandar** too.

In [ ]:
sns.set_style("darkgrid")

plt.figure(figsize=(10,6))
sns.lineplot(data=all_list_hive_total, x="Year", y="Amount", hue="Test_Smell_Type", linewidth=2.5, palette="bright")
plt.title("Test Smell Types by Year", fontsize=16)
plt.xlabel("Year", fontsize=14)
plt.ylabel("Amount", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
feature = ['NumberOfMethods', 'Assertion Roulette',
       'Conditional Test Logic', 'Constructor Initialization', 'Default Test',
       'EmptyTest', 'Exception Catching Throwing', 'General Fixture',
       'Mystery Guest', 'Print Statement', 'Redundant Assertion',
       'Sensitive Equality', 'Verbose Test', 'Sleepy Test', 'Eager Test',
       'Lazy Test', 'Duplicate Assert', 'Unknown Test', 'IgnoredTest',
       'Resource Optimism', 'Magic Number Test', 'Dependent Test']
list(enumerate(feature))

In [ ]:
for i in enumerate(feature):
    test_smell_show_inEachClass(i[1])

In [ ]:
plt.figure(figsize=(10, 7))
g = sns.FacetGrid(all_list_hive_total, col="Year")
g.map(sns.scatterplot, 'Test_Smell', 'Amount')
for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

In [ ]:
plt.figure(figsize= (15,30))
for i in enumerate(feature):
    sns.relplot(data=all_list_hive_total, x='Year', y=i[1])